# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project 2: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

**Answer: ** Students are separated into two categories for this dataset: those who passed and those who did not pass. This is therefore a discrete, categorical entity and not continuous. Thus, this is a classification task. 

## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [3]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"

Student data read successfully!


### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [4]:
# TODO: Calculate number of students
n_students = len(student_data)

# TODO: Calculate number of features
n_features = len(student_data.columns) - 1 # the final column is our set of labels, and not a feature

# TODO: Calculate passing students
n_passed = len(student_data[student_data["passed"] == "yes"])

# TODO: Calculate failing students
n_failed = len(student_data[student_data['passed'] == 'no'])

# TODO: Calculate graduation rate
grad_rate = float(n_passed)/float(n_students)*100

# Print the results
print "Total number of students: {}".format(n_students)
print "Number of features: {}".format(n_features)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of features: 30
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [5]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print "\nFeature values:"
print X_all.head()

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [6]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [7]:
# TODO: Import any additional functionality you may need here
from sklearn.cross_validation import train_test_split

# TODO: Set the number of training points
num_train = 300

# Set the number of testing points
num_test = X_all.shape[0] - num_train

# TODO: Shuffle and split the dataset into the number of training and testing points above
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, train_size = num_train, random_state = 0)

# Show the results of the split
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])

Training set has 300 samples.
Testing set has 95 samples.


## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

**The following supervised learning models are currently available in** [`scikit-learn`](http://scikit-learn.org/stable/supervised_learning.html) **that you may choose from:**
- Gaussian Naive Bayes (GaussianNB)
- Decision Trees
- Ensemble Methods (Bagging, AdaBoost, Random Forest, Gradient Boosting)
- K-Nearest Neighbors (KNeighbors)
- Stochastic Gradient Descent (SGDC)
- Support Vector Machines (SVM)
- Logistic Regression

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. For each model chosen*
- Describe one real-world application in industry where the model can be applied. *(You may need to do a small bit of research for this — give references!)* 
- What are the strengths of the model; when does it perform well? 
- What are the weaknesses of the model; when does it perform poorly?
- What makes this model a good candidate for the problem, given what you know about the data?

**Answer: ** Three supervised learning models that are appropriate for this problem: Decision Tree, K-Nearest Neighbours, Support Vector Machines.


DECISION TREE CLASSIFIER 
A real world application where the model can be applied:

One real world application of Decision Trees are Diagnosing Heart Disease patients. According to the paper "Using Decision Tree for Diagnosing Heart Disease Patients" by Shouman, Turner and Stocker, most research has applied decision trees based on Gain Ratio and binary discretization. Gini Index and Information Gain are two other successful types of Decision Trees that are less used in the diagnosis of heart disease.To evaluate the performance of the alternative Decision Trees, the sensitivity, specificity and accuracy are calculated.
The paper can be found [here](http://crpit.com/confpapers/CRPITV121Shouman.pdf).

Strengths of the model; When does it perform well?

Decision trees are easy to use and understand. They also do not require the data to have zero mean and unit variance, and can tolerate missing values in the features of the data. For explanatory purposes, they are easily visualized as flow charts. Decision trees support multi-output tasks and a single decision tree can be used for multiclass classification without employing a strategy like one-versus-all. They are eager learners and can predict relatively quickly once the model has been built. Decision trees also do not expect linear features and work well with or without them. According to the Decision Tree documentation on sklearn, the cost of predicting data is logarithmic in the number of data points used to train the tree, they can handle both numerical and categorical data, the model can be validated using statistical tests, and they perform well even if its assumptions are somewhat violated by the true model from which the data were generated.

Weaknesses of the model: When does it perform poorly?

Decision trees are more prone to overfitting as their learning algorithms can produce large, complicated decision trees that perfectly model every training instance but can fail to generalize the real relationship.This can be mitigated by pruning or creating a random forest. Additionally, efficient decision tree learning algorithms like ID3 are greedy and learn efficiently by making locally optimal decisions, but are not guaranteed to produce the globally optimal tree. According to the documentation on sklearn's decision tree implementation, they can be unstable because small variations in the data can result in a completely different tree and they create biased trees if some classes dominate.


What makes this model a good candidate for the problem given what you know about the data?

The student data may not be linearly separable and it has many features. Decision trees perform well in such situations while requiring relatively little effort for data preparation. Furthermore, decision trees, as eager learners, are well suited for making predictions on new student data in the future as it is predicts quickly. 



SUPPORT VECTOR MACHINES 
A real world application where the model can be applied:

As seen in the Academic paper "Text Categorization with Suppor Vector Machines: Learning with many relevant features" by Thorsten Joachims, SVMs are explored as a method for learning text classifiers from examples.
The paper can be found [here](http://link.springer.com/chapter/10.1007/BFb0026683).

Strengths of the model; when does it perform well?

In Support Vector Machines, the optimization objective is to maximize the margin, i.e. the difference between the separating hyperplane and the training samples that are closest to this hyperplane. SVMs are popular because they can be easily kernelized to solve nonlinear classification problems, making them effective in high dimensional spaces. This also allows the user to add domain knowledge via the kernel. Another useful property of support vector machines is that it has a single local minimum that is also the global  minimum. According to the documentation on SVMs in sklearn, they are effective in high-dimensional spaces, still effective when the number of dimensions is greater than the number of samples and uses a subset of training points making it very memory efficient.


Weaknesses of the model: When does it perform poorly?

Using SVM on datasets with a lot of features, where the number of features is greater than the number of samples is likely to give a poor performance as SVMs consider interaction between all features. Additionally, non-linear SVMs have a training complexity between O(n^2) and O(n^3) where n is the number of training examples, therefore in situations where they is a large dataset SVMs will not be able to train on all the data leading to poor performance, despite having a lot of data. 

What makes this model a good candidate for the problem, given what you know about the data?

The student dataset is relatively small, making Support Vector Machines a good choice of algorithm overall. Due to SVM's versatility, it will not matter whether the data is linearly seperable or not as this algorithm can support both types of data. 

K-NEAREST NEIGHBOURS

A real world application where the model can be applied:

Gene expression as shown in the paper "Gene Assessment and Sample Classification for Gene Expression Data Using a Genetic Algorithm / k-nearest Neighbor Method" by Li, L.; Darden, T.A.; Weingberg, C.R.; Levine, A.J.; Pedersen, L.G. A combination of a genetic algorithm and KNN method is used to identify genes that jointly can discriminate between two types of samples, for example normal versus tumour.
The paper can be found [here](http://www.ingentaconnect.com/content/ben/cchts/2001/00000004/00000008/art00010).

Strengths of the model: when does it perform well?

The K-Nearest Neighbours algorithm takes no time to train data and so the cost of the learning process is zero. Because of this, no assumptions about the data, i.e. what concepts might be learned are made. It also keeps all the original data and works very well with large datasets. Additionally, the concepts or rules are that learned are learned by local approximation using simple procedures, thus adhering to Occam's Razor in that it prefers simpler models. For explanatory purposes, this is a very simple algorithm to explain and can be understood with relatively little knowledge of machine learning.

Weaknesses of the model: When does it perform poorly?

While the algorithm takes no time to learn that data, it is computationally expensive to find an appropriate K when the dataset is very large. Furthermore, the concepts that are learned cannot be interpreted as no description is given of them. Care must be taken to remove irrelevant or correlated features as the nearest neighbour is easily misled by noisy features. This might also lead to overfitting the model. Finally, the performance of the algorithm depends on the number of dimensions of the data as the "neighbourhood" then becomes very large and the algorithm suffers from the curse of dimensionality.

What makes this model a good candidate for the problem, given what you know about the data?

The student dataset is very small and finding K would be computationally inexpensive. The KNN algorithm is a lazy learner and can easily be adapted to a larger student population in the future. 

### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [8]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print "Trained model in {:.4f} seconds".format(end - start)

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print "Made predictions in {:.4f} seconds.".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print "Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print "F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test))

### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [9]:
# TODO: Import the three supervised learning models from sklearn
from sklearn.tree import DecisionTreeClassifier 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

# TODO: Initialize the three models
clf_A = DecisionTreeClassifier(random_state = 0)
clf_B = KNeighborsClassifier()
clf_C = SVC(random_state = 0)

# TODO: Set up the training set sizes
X_train_100 = X_train[:100]
y_train_100 = y_train[:100]

X_train_200 = X_train[:200]
y_train_200 = y_train[:200]

X_train_300 = X_train
y_train_300 = y_train

# TODO: Execute the 'train_predict' function for each classifier and each training set size
train_predict(clf_A, X_train_100, y_train_100, X_test, y_test)
train_predict(clf_A, X_train_200, y_train_200, X_test, y_test)
train_predict(clf_A, X_train_300, y_train_300, X_test, y_test)

train_predict(clf_B, X_train_100, y_train_100, X_test, y_test)
train_predict(clf_B, X_train_200, y_train_200, X_test, y_test)
train_predict(clf_B, X_train_300, y_train_300, X_test, y_test)

train_predict(clf_C, X_train_100, y_train_100, X_test, y_test)
train_predict(clf_C, X_train_200, y_train_200, X_test, y_test)
train_predict(clf_C, X_train_300, y_train_300, X_test, y_test)

Training a DecisionTreeClassifier using a training set size of 100. . .
Trained model in 0.0310 seconds
Made predictions in 0.0040 seconds.
F1 score for training set: 1.0000.
Made predictions in 0.0000 seconds.
F1 score for test set: 0.6942.
Training a DecisionTreeClassifier using a training set size of 200. . .
Trained model in 0.0050 seconds
Made predictions in 0.0010 seconds.
F1 score for training set: 1.0000.
Made predictions in 0.0010 seconds.
F1 score for test set: 0.7132.
Training a DecisionTreeClassifier using a training set size of 300. . .
Trained model in 0.0080 seconds
Made predictions in 0.0010 seconds.
F1 score for training set: 1.0000.
Made predictions in 0.0010 seconds.
F1 score for test set: 0.7167.
Training a KNeighborsClassifier using a training set size of 100. . .
Trained model in 0.0400 seconds
Made predictions in 0.0150 seconds.
F1 score for training set: 0.7972.
Made predictions in 0.0040 seconds.
F1 score for test set: 0.7068.
Training a KNeighborsClassifier us

### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

** Classifer 1 - Decision Tree**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |       0.0040        |        0.0000          |        1         |      0.6942     |
| 200               |         0.0050          |        0.0010          |        1         |      0.7132     |
| 300               |         0.0090          |        0.0010          |        1         |      0.7167     |

** Classifer 2 - K-Nearest Neighbours**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |        0.0030           |       0.0050           |     0.7972       |     0.7068      |
| 200               |        0.0020           |       0.0050           |     0.8571       |     0.7482      |
| 300               |        0.0020           |       0.0070           |     0.8722       |     0.7838      |

** Classifer 3 - Support Vector Machines**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |       0.0040            |      0.0020            |      0.8591      |     0.7838      |
| 200               |        0.0110           |      0.0040            |      0.8693      |     0.7755      |
| 300               |        0.0210           |      0.0050            |      0.8692      |     0.7586      |

## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Choosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

**Answer: ** The student dataset has 395 observations and 30 features. This is small dataset and will likely have trouble predicting whether a student requires intervention. All three of the models chosen have prediction and training times of under 0.5 seconds, however the KNN algorithm has the fastest training time and the Decision Tree algorithm has the fastest prediction time. The decision tree also had the lowest F1 score, hardly better than random guessing in fact as a model that predicts every student will pass will have a similar accuracy (0.67 based on the graduation score). The Support Vector Machines on the other hand, had the highest f1 scores on each iteration for both training and test data and also performed in under 0.5 seconds, making it the best performing algorithm of the three. Furthermore, the difference between the F1 score at 200 training observations and 300 is very small while the training time almost doubles. Taking these factors into consideration, I would choose the SVM as both the best model and the most appropriate model based on the available data, limited resources, cost and performance. 

### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. Be sure that you are describing the major qualities of the model, such as how the model is trained and how the model makes a prediction. Avoid using advanced mathematical or technical jargon, such as describing equations or discussing the algorithm implementation.*

**Answer: ** The final model, Support Vector Machines or SVM, separated the data we have into the two classes (students who passed and students who did not pass) by placing a separating line at the point where the distance between the classes is the greatest as shown in the following graph. 

![Support Vectors](https://udacity-github-sync-content.s3.amazonaws.com/_imgs/372/1472304887/SVM_2.png)

When the data cannot be easily separated by a straight line, the algorithm is able to project the data onto a "higher-dimensional" feature space. Ordinarily this would be very computationally expensive, however the SVM can use a formula known as a kernel function to implicitly work in a higher dimensional space without explicitly building the higher-dimensional space where it is then able to find an appropriate line to separate the data as shown in the following visualizations. 

![The Kernel Trick](https://udacity-github-sync-content.s3.amazonaws.com/_imgs/372/1472304892/data_2d_to_3d_hyperplane.png)

The model learns "rules" to determine how the features (for example family size) relate to the result, i.e. whether a student passes or not, based on this separating line. When the model has learned the where the data can best be separated, any new student data can be inputed into it and based on the rules learned, it can determine how likely it is that the student will graduate. 

### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.gridSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [11]:
# TODO: Import 'GridSearchCV' and 'make_scorer'
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import make_scorer

# TODO: Create the parameters list you wish to tune
parameters = {'kernel':['linear', 'rbf', 'poly', 'sigmoid'], "C":[pow(2, c) for c in range(-3, 3)], "gamma":[pow(2, g) for g in range(-3, 3)], "degree":[d for d in range(1, 10)]}

# TODO: Initialize the classifier
clf = SVC(random_state = 0)

# TODO: Make an f1 scoring function using 'make_scorer' 
f1_scorer = make_scorer(f1_score, pos_label='yes')

# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
grid_obj = GridSearchCV(clf, parameters, scoring = f1_scorer)

# TODO: Fit the grid search object to the training data and find the optimal parameters
grid_obj.fit(X_train, y_train)

# Get the estimator
clf = grid_obj.best_estimator_

# Report the final F1 score for training and testing after parameter tuning
print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test))

Made predictions in 0.0080 seconds.
Tuned model has a training F1 score of 0.8351.
Made predictions in 0.0030 seconds.
Tuned model has a testing F1 score of 0.7945.


### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

**Answer: ** The final model's F1 training score is 0.8351 and the F1 testing score is 0.7945. This is a significant improvement in accuracy on the untuned model's F1 testing score of 0.7586.

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.